# <h1 style="text-align: left; color: #FFB347; font-family: 'Bahnschrift Condensed', sans-serif; font-size: 2.5em;">Connecting to MySQL Workbench
</h1>


In [257]:
# 📚 Basic libraries
import pandas as pd # data manipulation
import numpy as np # numerical operations
import warnings # warning messages management
import matplotlib.pyplot as plt # plotting
import seaborn as sns # plotting
colors = ['#FF7F7F', '#FFF9C4', '#A5D6A7', '#81D4FA', '#F1F1F1']

# 📊 Statistical libraries
from scipy.stats import chi2_contingency # 🧮 Chi-squared test for independence
from scipy.stats.contingency import association # 📈 Measures association strength between categorical variables
import scipy.stats as st # ⚙️ General statistical functions
import statsmodels.api as sm # 📐 Advanced statistical models and testing
import statsmodels.formula.api as smf # 📝 Formula-based interface for statistical models

# Set the color palette for all plots by default
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=colors)
# ⚙️ Settings
pd.set_option('display.max_columns', None) # display all columns
warnings.filterwarnings('ignore') # ignore warnings

# 🌐 New Libraries
from getpass import getpass # get password without showing it
import pymysql # MySQL connection
import requests # HTTP requests
import time # time operations

# ⚙️ Settings
pd.set_option('display.max_columns', None) # display all columns
warnings.filterwarnings('ignore') # ignore warnings

def create_connection(): # create a connection to a MySQL database
    """
    Create a connection to a MySQL database.
    Prompts the user for the database password and database name.
    
    Returns:
        cnx: pymysql.connections.Connection object
            The connection object to the MySQL database.
    """
    password = getpass("Please, kindly insert your password: ")
    database = input("Please, kindly insert your database name: ")
    cnx = pymysql.connect(user='root', password=password,
                          host='localhost', database=database)
    if cnx.open:
        print("Connection open")
    else:
        print("Connection is not successfully open")
    return cnx

In [258]:
# We enter the password and the database 
connection = create_connection()

Connection open


In [259]:
df_user_filter=pd.read_csv("../data/df_user_filter.csv")
df_user_filter_collapsed=pd.read_csv("../data/df_user_filter_collapsed.csv")
df_location_filter=pd.read_csv("../data/df_location_filter.csv")
df_location_filter_collapsed=pd.read_csv("../data/df_location_filter_collapsed.csv")
query="SELECT * FROM rating;"
df_rating=pd.read_sql(query,connection)

# <h1 style="text-align: left; color: #FFB347; font-family: 'Bahnschrift Condensed', sans-serif; font-size: 2.5em;">Formating Rating Dataframe prior to merge
</h1>


In [260]:
df_location_filter.head(1)

,Place ID,R Latitude,R Longitude,Restaurant Name,Address,City,State,Country,ZIP Code,Alcohol Availability,Smoking Area,Dress Code,Accessibility,Price Range,Ambience,Area,Other Services,Cuisine Type,Parking Availability,Payment Methods,Opening Hours,Operating Days
0,135109,18.921785,-99.23535,Paniroles,?,NaN,NaN,Mexico,?,Wine/ Beer,Not Permitted,Informal,No Accessibility,Medium,Quiet,Closed,Internet,Italian,NaN,Cash,08:00-21:00;,Mon;Tue;Wed;Thu;Fri;


In [261]:
df_user_filter.head(1)

,User ID,Latitude,Longitude,Smoking Habit,Drinking Habit,Dress Preference,Preferred Ambience,Transport Mode,Marital Status,People Under Charge,Birth Year,Interest,Personality,Religion,Activity Level,Favorite Color,Weight (kg),Budget,Height (cm),Preferred Payment Method,User Preference
0,U1001,22.139997,-100.978803,False,Abstemious,Informal,Family,On foot,Single,Independent,1989,Variety,Thrifty protector,Atheist,Student,Black,69,Medium,1.77,Cash,American


In [262]:
df_rating.head(1)

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2


In [263]:
rename_columns = {
    "userID": "User ID",
    "placeID": "Place ID",
    "rating": "Rating",
    "food_rating": "Food Rating",
    "service_rating": "Service Rating"
}


df_rating.rename(columns=rename_columns, inplace=True)

# <h1 style="text-align: left; color: #FFB347; font-family: 'Bahnschrift Condensed', sans-serif; font-size: 2.5em;">NaN Handling
</h1>


In [264]:
df_user_filter["Interest"]=df_user_filter["Interest"].fillna(method="ffill")

In [265]:
df_user_filter_collapsed["Interest"]=df_user_filter_collapsed["Interest"].fillna(method="ffill")

In [266]:
df_location_filter_collapsed["City"]=df_location_filter_collapsed["City"].fillna(method="ffill")
df_location_filter_collapsed["City"]=df_location_filter_collapsed["City"].fillna(method="bfill")
df_location_filter_collapsed["State"]=df_location_filter_collapsed["State"].fillna(method="ffill")
df_location_filter_collapsed["State"]=df_location_filter_collapsed["State"].fillna(method="bfill")
df_location_filter_collapsed["Smoking Area"]=df_location_filter_collapsed["Smoking Area"].fillna(method="ffill")
df_location_filter_collapsed["Other Services"]=df_location_filter_collapsed["Other Services"].fillna(method="ffill")
df_location_filter_collapsed["Parking Availability"]=df_location_filter_collapsed["Parking Availability"].fillna(method="ffill")
df_location_filter_collapsed["Parking Availability"]=df_location_filter_collapsed["Parking Availability"].fillna(method="bfill")


In [267]:
df_location_filter["City"] = df_location_filter["City"].fillna(method="ffill")
df_location_filter["City"] = df_location_filter["City"].fillna(method="bfill")
df_location_filter["State"] = df_location_filter["State"].fillna(method="ffill")
df_location_filter["State"] = df_location_filter["State"].fillna(method="bfill")
df_location_filter["Smoking Area"] = df_location_filter["Smoking Area"].fillna(method="ffill")
df_location_filter["Other Services"] = df_location_filter["Other Services"].fillna(method="ffill")
df_location_filter["Parking Availability"] = df_location_filter["Parking Availability"].fillna(method="ffill")
df_location_filter["Parking Availability"] = df_location_filter["Parking Availability"].fillna(method="bfill")


# <h1 style="text-align: left; color: #FFB347; font-family: 'Bahnschrift Condensed', sans-serif; font-size: 2.5em;">Merging Dataframes
</h1>


In [268]:
df_users=pd.merge(df_rating,df_user_filter,on=["User ID"],how="inner")
df_filter=pd.merge(df_location_filter,df_users,on=["Place ID"], how="inner")

In [269]:
df_filter.isna().sum()

Place ID                    0
R Latitude                  0
R Longitude                 0
Restaurant Name             0
Address                     0
City                        0
State                       0
Country                     0
ZIP Code                    0
Alcohol Availability        0
Smoking Area                0
Dress Code                  0
Accessibility               0
Price Range                 0
Ambience                    0
Area                        0
Other Services              0
Cuisine Type                0
Parking Availability        0
Payment Methods             0
Opening Hours               0
Operating Days              0
User ID                     0
Rating                      0
Food Rating                 0
Service Rating              0
Latitude                    0
Longitude                   0
Smoking Habit               0
Drinking Habit              0
Dress Preference            0
Preferred Ambience          0
Transport Mode              0
Marital St

In [270]:
df_users_collapsed=pd.merge(df_rating, df_user_filter_collapsed, on=["User ID"], how="inner")
df_filter_collapsed=pd.merge(df_location_filter_collapsed,df_users_collapsed,on=["Place ID"],how="inner")

In [271]:
df_filter_collapsed.shape

(781, 46)

We will keep the datasets with the ratings as digits, in case we want to use them in a machine learning predictive model. However, for now, we will convert them to 'strings' for the filtering model (restaurant recommendation).

In [272]:
df_filter_collapsed.to_csv("../data/df_filter_collapsed_ratingdigit.csv",index=False)

In [273]:
df_filter.to_csv("../data/df_filter_ratingdigit.csv",index=False)

## <h1 style="text-align: left; color: #FFB347; font-family: 'Bahnschrift', sans-serif; font-size: 1.5em;">Changing column location names</h1>


In [274]:
df_filter = df_filter.rename(columns={
    'R Latitude': 'latitude',
    'R Longitude': 'longitude',
    'Latitude': 'U latitude',
    'Longitude': 'U longitude'
})

df_filter_collapsed = df_filter_collapsed.rename(columns={
    'R Latitude': 'latitude',
    'R Longitude': 'longitude',
    'Latitude': 'U latitude',
    'Longitude': 'U longitude'
})

In [280]:
df_filter_collapsed['Restaurant Name'] = df_filter_collapsed['Restaurant Name'].str.replace(r'.*N De San Francisco.*', 'El Rincon De San Francisco', regex=True)
df_filter['Restaurant Name'] = df_filter['Restaurant Name'].str.replace(r'.*N De San Francisco.*', 'El Rincon De San Francisco', regex=True)

## <h1 style="text-align: left; color: #FFB347; font-family: 'Bahnschrift', sans-serif; font-size: 1.5em;">Mapping Rating values</h1>


In [177]:
df_filter_collapsed.replace({0: 'Dissatisfied', 1: 'Neutral', 2: 'Satisfied'}, inplace=True)

In [276]:
df_filter.replace({0: 'Dissatisfied', 1: 'Neutral', 2: 'Satisfied'}, inplace=True)

In [99]:
##df_filter_collapsed.to_csv("../data/df_filter_collapsed.csv",index=False)

In [ ]:
##df_filter.to_csv("../data/df_filter.csv",index=False)